<a href="https://colab.research.google.com/github/rulas99/thermal_anomaly_detection/blob/main/thermal_anomalies_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Semantic Segmentation [source](https://towardsdatascience.com/train-neural-net-for-semantic-segmentation-with-pytorch-in-50-lines-of-code-830c71a6544f)

In [ ]:
import os
import numpy as np
import cv2
import torchvision.models.segmentation
import torch
import torchvision.transforms as tf

from pandas import read_csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
Learning_Rate = 1e-5
width = 500
height = 500 # image width and height
batchSize = 7

In [ ]:
modelPath = "/content/drive/MyDrive/Inge/segment_sentinel2_hotspot/Models_2/model_950.torch"

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

Net = torchvision.models.segmentation.deeplabv3_resnet50(pretrained=True) # Load net

Net.classifier[4] = torch.nn.Conv2d(256, 2, kernel_size=(1, 1), stride=(1, 1)) # Change final layer to 2 classes

Net = Net.to(device) # Set net to GPU or CPU

#Net.load_state_dict(torch.load(modelPath)) # Load trained model

optimizer = torch.optim.Adam(params=Net.parameters(),lr=Learning_Rate) # Create adam optimizer

weights = torch.FloatTensor([0.01,1]).cuda()
criterion = torch.nn.CrossEntropyLoss(weight = weights) # Set loss function

Downloading: "https://download.pytorch.org/models/deeplabv3_resnet50_coco-cd0a2569.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet50_coco-cd0a2569.pth


  0%|          | 0.00/161M [00:00<?, ?B/s]

In [ ]:
train=read_csv("/content/drive/MyDrive/segment_sentinel2_hotspot/train_2.csv")

In [ ]:
transformImg=tf.Compose([tf.ToPILImage(),tf.Resize((height,width)),tf.ToTensor(),])
                         #tf.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])
                         
transformAnn=tf.Compose([tf.ToPILImage(),tf.Resize((height,width),tf.InterpolationMode.NEAREST),tf.ToTensor()])

In [ ]:
def ReadRandomImage(): # First lets load random image and  the corresponding annotation
    idx=np.random.randint(0,len(train)) # Select random image
    thermal = cv2.imread(train.ori_path.iloc[idx])
    masked =  cv2.imread(train.label_path.iloc[idx],0)
    AnnMap = np.zeros(thermal.shape[0:2],np.float32)
    if masked is not None:  AnnMap[ masked == 255 ] = 1
    Img=transformImg(thermal)
    AnnMap=transformAnn(AnnMap)
    return Img,AnnMap


def LoadBatch(): # Load batch of images
    images = torch.zeros([batchSize,3,height,width])
    ann = torch.zeros([batchSize, height, width])
    for i in range(batchSize):
        images[i],ann[i]=ReadRandomImage()
    return images, ann

In [ ]:
for itr in range(3000): # Training loop
   images,ann=LoadBatch() # Load taining batch
   images=torch.autograd.Variable(images,requires_grad=False).to(device) # Load image
   ann = torch.autograd.Variable(ann, requires_grad=False).to(device) # Load annotation
   Pred=Net(images)['out'] # make prediction
   optimizer.zero_grad()
   Loss=criterion(Pred,ann.long()) # Calculate cross entropy loss
   Loss.backward() # Backpropagate loss
   optimizer.step() # Apply gradient descent change to weight

   loss = round(float(Loss.data.cpu()),5)

   if itr % 100 == 0 or loss<0.06: #Save model weight once every 100 steps permenant file
        #accG = [(torch.argmax(Pred[i], 0).cpu().detach().numpy() == ann[i].cpu().detach().numpy()).sum()/(width*height) for i in range(batchSize)]
        #accM = round(sum(accG)/batchSize,5)
        print(itr,f") Loss = {loss}")
        print(f"Saving model_{itr}.torch")
        torch.save(Net.state_dict(),   f'/content/drive/MyDrive/segment_sentinel2_hotspot/Models_2/model_{itr}.torch')

        if loss<0.06:
          break

0 ) Loss = 0.65564
Saving model_0.torch
100 ) Loss = 0.45739
Saving model_100.torch
200 ) Loss = 0.40238
Saving model_200.torch
300 ) Loss = 0.2337
Saving model_300.torch
400 ) Loss = 0.24082
Saving model_400.torch
500 ) Loss = 0.20281
Saving model_500.torch
600 ) Loss = 0.15643
Saving model_600.torch
700 ) Loss = 0.17615
Saving model_700.torch
800 ) Loss = 0.16729
Saving model_800.torch
900 ) Loss = 0.14873
Saving model_900.torch
1000 ) Loss = 0.1553
Saving model_1000.torch
1100 ) Loss = 0.12879
Saving model_1100.torch
1200 ) Loss = 0.11952
Saving model_1200.torch
1300 ) Loss = 0.1121
Saving model_1300.torch
1400 ) Loss = 0.1034
Saving model_1400.torch
1500 ) Loss = 0.09503
Saving model_1500.torch
1600 ) Loss = 0.09392
Saving model_1600.torch
1700 ) Loss = 0.07742
Saving model_1700.torch
1800 ) Loss = 0.08402
Saving model_1800.torch
1900 ) Loss = 0.07929
Saving model_1900.torch
2000 ) Loss = 0.06727
Saving model_2000.torch
2030 ) Loss = 0.05897
Saving model_2030.torch
